In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import xml.etree.ElementTree as et
from io import BytesIO
from zipfile import ZipFile
from bs4 import BeautifulSoup as bs
import xmltodict
import json
import dart_fss as dart
from lxml import html
from xml.etree.ElementTree import parse
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import numpy as np
import datetime
try:
    from pandas import json_normalize
except ImportError:
    from pandas.io.json import json_normalize
import OpenDartReader
from bs4 import BeautifulSoup
import FinanceDataReader as fdr
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
kospi_list = pd.read_excel('kospi.xlsx', index_col=1)

In [3]:
crtfc_key = '022d222dfd753fee87cfafad7fe54169c4d72eb9'
def get_corpcode(crtfc_key): 
    """ 
    OpenDART 기업 고유번호 받아오기 
    return 값: 주식코드를 가진 업체의 DataFrame 
    """ 
    params = {'crtfc_key':crtfc_key} 
    items = ["corp_code","corp_name","stock_code","modify_date"] 
    item_names = ["고유번호","회사명","종목코드","수정일"] 
    url = "https://opendart.fss.or.kr/api/corpCode.xml" #요청 url
    response = requests.get(url,params=params) #url 불러오기
    zfile = ZipFile(io.BytesIO(response.content))
    fin = zfile.open(zfile.namelist()[0])  #zip file 열고
    root = et.fromstring(fin.read().decode('utf-8'))  #utf-8 디코딩

    data = [] 
    for child in root: 
        if len(child.find('stock_code').text.strip()) > 1: # 종목코드가 있는 경우 
            data.append([]) #data에 append 
            for item in items: 
                data[-1].append(child.find(item).text) 
    df = pd.DataFrame(data, columns=item_names) 
    return df


stock_comp = get_corpcode(crtfc_key)

with open('고유번호.txt', 'w', encoding='utf-8') as file:
    file.write(stock_comp.to_string())


# 20200101으로 나오는 기존 date를 2020-01-01 형식으로 변형
stock_comp['수정일'] = pd.to_datetime(stock_comp['수정일'], format='%Y%m%d')

stock_comp

,고유번호,회사명,종목코드,수정일
0,00260985,한빛네트,036720,2017-06-30
1,00264529,엔플렉스,040130,2017-06-30
2,00358545,동서정보기술,055000,2017-06-30
3,00231567,애드모바일,032600,2017-06-30
4,00247939,씨모스,037600,2017-06-30
...,...,...,...,...
3710,00330044,캠시스,050110,2024-04-01
3711,00661847,화인베스틸,133820,2024-04-01
3712,00346407,큐로컴,040350,2024-04-01
3713,01428203,케이씨씨글라스,344820,2024-04-01


In [4]:
key = "022d222dfd753fee87cfafad7fe54169c4d72eb9" 
url = f"http://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={key}"
res_url = requests.get(url)

ZipFile(BytesIO(res_url.content)).extractall("./data")
xml = open("./data/CORPCODE.xml", "r", encoding='utf-8').read()
dict_type = xmltodict.parse(xml)

corp_dict = {}
for company in dict_type['result']['list']:
    if company['stock_code']:
        corp_dict[int(company['stock_code'])] = company['corp_code']
        
kospi_list['고유번호'] = None
for idx in kospi_list.index:
    kospi_list.loc[idx, '고유번호'] = corp_dict[idx]

In [5]:
# 기업별 고유번호 포함한 데이터프레임 저장
kospi_list.reset_index(inplace=True)
kospi_list.set_index('고유번호', drop=True, inplace=True)
kospi_list.to_excel('kospi.xlsx')
kospi_list

,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
고유번호,,,,,,,,,
00155212,22100,포스코DX,"건축기술, 엔지니어링 및 관련 기술 서비스업",EIC 자동화 엔지니어링 및 IT서비스,45293,12월,정덕균,http://www.poscodx.com/kor/,경상북도
01311408,450080,에코프로머티,일차전지 및 축전지 제조업,배터리 양극재용 핵심소재 하이니켈 전구체,45247,12월,김병훈,http://ecopromaterials.com/,경상북도
01515323,373220,LG에너지솔루션,일차전지 및 축전지 제조업,"2차전지 (소형,ESS,자동차전지)",44588,12월,김동명,NaN,서울특별시
01596425,402340,SK스퀘어,기타 금융업,지주회사,44529,12월,박성하,http://www.sksquare.com,서울특별시
01244601,377300,카카오페이,금융 지원 서비스업,"간편결제, 간편송금 및 기타 금융서비스",44503,12월,신원근,http://www.kakaopay.com,경기도
...,...,...,...,...,...,...,...,...,...
00113207,1440,대한전선,절연선 및 케이블 제조업,"전력선,통신케이블,적산계기,스텐레스압연제품,광케이블,초고압선,알루미늄 제조,도매/전기공사",25199,12월,송종민,http://www.taihan.com,경기도
00160047,240,한국앤컴퍼니,기타 금융업,비금융지주회사,25199,12월,"조현범, 안종선(각자 대표이사)",http://www.hankook-technologygroup.com,경기도
00113526,3490,대한항공,항공 여객 운송업,"여객운송,화물운송,항공기 제조,판매,정비수리/기내식 제조판매,면세품 판매",24184,12월,"조원태, 우기홍",http://www.koreanair.com,서울특별시


### URL1 최대주주 성공

In [7]:
import json
import requests

key = "022d222dfd753fee87cfafad7fe54169c4d72eb9"
year = 2023

# DataFrame 초기화
kospi_list['최대주주_성명'], kospi_list['최대주주_관계'], kospi_list['최대주주_주식종류'] = None, None, None
kospi_list['최대주주_기초주식수'], kospi_list['최대주주_기초주식지분율'] = None, None
kospi_list['최대주주_기말주식수'], kospi_list['최대주주_기말주식지분율'] = None, None

for idx in kospi_list.index:
    res_url_1 = requests.get(f'https://opendart.fss.or.kr/api/hyslrSttus.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011', headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    })    
    
    json_reports = json.loads(res_url_1.text)
    
    if json_reports.get('list'):
        for company_report in json_reports['list']:
            if company_report.get('relate') == "최대주주":
                # 최대주주 정보만 가져오기
                kospi_list.loc[idx, '최대주주_성명'] = company_report.get('nm', None)
                kospi_list.loc[idx, '최대주주_관계'] = company_report.get('relate', None)
                kospi_list.loc[idx, '최대주주_주식종류'] = company_report.get('stock_knd', None)
                kospi_list.loc[idx, '최대주주_기초주식수'] = company_report.get('bsis_posesn_stock_co', None)
                kospi_list.loc[idx, '최대주주_기초주식지분율'] = company_report.get('bsis_posesn_stock_qota_rt', None)
                kospi_list.loc[idx, '최대주주_기말주식수'] = company_report.get('trmend_posesn_stock_co', None)
                kospi_list.loc[idx, '최대주주_기말주식지분율'] = company_report.get('trmend_posesn_stock_qota_rt', None)


In [8]:
kospi_list

,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,사업자등록번호,주소,업종코드,설립일,최대주주_성명,최대주주_관계,최대주주_주식종류,최대주주_기초주식수,최대주주_기초주식지분율,최대주주_기말주식수,최대주주_기말주식지분율
고유번호,,,,,,,,,,,,,,,,,,,,
00155212,22100,포스코DX,"건축기술, 엔지니어링 및 관련 기술 서비스업",EIC 자동화 엔지니어링 및 IT서비스,45293,12월,정덕균,http://www.poscodx.com/kor/,경상북도,2198100428,경상북도 포항시 남구 호동로 68,72129,19891118,포스코홀딩스,최대주주,보통주,"99,403,282",65.38,"99,403,282",65.38
01311408,450080,에코프로머티,일차전지 및 축전지 제조업,배터리 양극재용 핵심소재 하이니켈 전구체,45247,12월,김병훈,http://ecopromaterials.com/,경상북도,8108800647,경상북도 포항시 북구 흥해읍 영일만산단남로75번길 15,28202,20170426,주식회사 에코프로,최대주주,보통주,"30,559,067",52.78,"30,559,067",44.30
01515323,373220,LG에너지솔루션,일차전지 및 축전지 제조업,"2차전지 (소형,ESS,자동차전지)",44588,12월,김동명,NaN,서울특별시,8518102050,"서울특별시 영등포구 여의대로 108, 타워 1",28202,20201201,None,None,None,None,None,None,None
01596425,402340,SK스퀘어,기타 금융업,지주회사,44529,12월,박성하,http://www.sksquare.com,서울특별시,5408102301,서울특별시 중구 을지로 65 SK T타워,64992,20211102,SK㈜,최대주주,의결권있는 주식,"42,452,202",30.01,"42,452,202",30.01
01244601,377300,카카오페이,금융 지원 서비스업,"간편결제, 간편송금 및 기타 금융서비스",44503,12월,신원근,http://www.kakaopay.com,경기도,5278800686,경기도 성남시 분당구 판교역로 166 카카오페이판교아지트 B동 15층,66199,20170403,㈜카카오,최대주주,보통주,"62,351,920",46.88,"62,351,920",46.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00113207,1440,대한전선,절연선 및 케이블 제조업,"전력선,통신케이블,적산계기,스텐레스압연제품,광케이블,초고압선,알루미늄 제조,도매/전기공사",25199,12월,송종민,http://www.taihan.com,경기도,1198110062,경기도 안양시 동안구 관양동 1746-2 대한스마트타워,28302,19550221,주식회사 호반산업,최대주주,보통주,"499,044,607",40.10,"49,904,460",40.10
00160047,240,한국앤컴퍼니,기타 금융업,비금융지주회사,25199,12월,"조현범, 안종선(각자 대표이사)",http://www.hankook-technologygroup.com,경기도,2208102784,경기도 성남시 분당구 판교로 286 -,64992,19410510,None,None,None,None,None,None,None
00113526,3490,대한항공,항공 여객 운송업,"여객운송,화물운송,항공기 제조,판매,정비수리/기내식 제조판매,면세품 판매",24184,12월,"조원태, 우기홍",http://www.koreanair.com,서울특별시,1108114794,서울특별시 강서구 하늘길 260,511,19620619,None,None,None,None,None,None,None


### URL2 소액주주 성공

In [7]:
import json
import requests

key = "022d222dfd753fee87cfafad7fe54169c4d72eb9"
year = 2023

kospi_list['소액주주수'], kospi_list['주주수'], kospi_list['소액주주비율'] = None, None, None
kospi_list['소액주주_주식수'], kospi_list['총주식수'], kospi_list['소액주주_주식지분율'] = None, None, None

for idx in kospi_list.index:
    res_url_2 = requests.get(f'https://opendart.fss.or.kr/api/mrhlSttus.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011',  
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'})    
    
    json_reports = json.loads(res_url_2.text)
    if json_reports.get('list'):
        for company_report in json_reports['list']:
            kospi_list.loc[idx, '소액주주수'] = company_report.get('shrholdr_co', None)
            kospi_list.loc[idx, '주주수'] = company_report.get('shrholdr_tot_co', None)
            kospi_list.loc[idx, '소액주주비율'] = company_report.get('shrholdr_rate', None)
            kospi_list.loc[idx, '소액주주_주식수'] = company_report.get('hold_stock_co', None)
            kospi_list.loc[idx, '총주식수'] = company_report.get('stock_tot_co', None)
            kospi_list.loc[idx, '소액주주_주식지분율'] = company_report.get('hold_stock_rate', None)


In [6]:
kospi_list

,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,소액주주수,주주수,소액주주비율,소액주주_주식수,총주식수,소액주주_주식지분율
고유번호,,,,,,,,,,,,,,,
00155212,22100,포스코DX,"건축기술, 엔지니어링 및 관련 기술 서비스업",EIC 자동화 엔지니어링 및 IT서비스,45293,12월,정덕균,http://www.poscodx.com/kor/,경상북도,"170,632","170,633",99.999%,"52,631,447","152,034,729",34.62%
01311408,450080,에코프로머티,일차전지 및 축전지 제조업,배터리 양극재용 핵심소재 하이니켈 전구체,45247,12월,김병훈,http://ecopromaterials.com/,경상북도,None,None,None,None,None,None
01515323,373220,LG에너지솔루션,일차전지 및 축전지 제조업,"2차전지 (소형,ESS,자동차전지)",44588,12월,김동명,NaN,서울특별시,-,-,-,-,-,-
01596425,402340,SK스퀘어,기타 금융업,지주회사,44529,12월,박성하,http://www.sksquare.com,서울특별시,"219,390","219,454",99.9%,"70,556,448","141,467,571",49.9%
01244601,377300,카카오페이,금융 지원 서비스업,"간편결제, 간편송금 및 기타 금융서비스",44503,12월,신원근,http://www.kakaopay.com,경기도,"306,557","306,560",99.99%,"16,619,200","131,883,080",12.60%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00113207,1440,대한전선,절연선 및 케이블 제조업,"전력선,통신케이블,적산계기,스텐레스압연제품,광케이블,초고압선,알루미늄 제조,도매/전기공사",25199,12월,송종민,http://www.taihan.com,경기도,"465,852","465,856",99.99%,"461,648,442","856,473,009",53.90%
00160047,240,한국앤컴퍼니,기타 금융업,비금융지주회사,25199,12월,"조현범, 안종선(각자 대표이사)",http://www.hankook-technologygroup.com,경기도,"22,746","22,762",99.93%,"22,297,182","94,935,240",23.49%
00113526,3490,대한항공,항공 여객 운송업,"여객운송,화물운송,항공기 제조,판매,정비수리/기내식 제조판매,면세품 판매",24184,12월,"조원태, 우기홍",http://www.koreanair.com,서울특별시,"878,262","888,423",98.85%,"202,463,650","348,931,619",58.02%


### URL3 감사의견 성공

In [9]:
import json
import requests

key = "022d222dfd753fee87cfafad7fe54169c4d72eb9"
year = 2023

# DataFrame 초기화
kospi_list['감사의견'], kospi_list['감사의견_강조사항'], kospi_list['감사의견_핵심감사사항'] = None, None, None

for idx in kospi_list.index:
    res_url_3 = requests.get(f'https://opendart.fss.or.kr/api/accnutAdtorNmNdAdtOpinion.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011', headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    })    
    
    json_reports = json.loads(res_url_3.text)
    
    if json_reports.get('list'):
        adt_opinion_list, emphs_matter_list, core_adt_matter_list = [], [], []
        for company_report in json_reports['list']:
            adt_opinion_list.append(company_report.get('adt_opinion', ''))
            emphs_matter_list.append(company_report.get('emphs_matter', ''))
            core_adt_matter_list.append(company_report.get('core_adt_matter', ''))

        # 리스트에 있는 값을 하나로 합쳐서 DataFrame에 할당
        kospi_list.loc[idx, '감사의견'] = ' '.join(adt_opinion_list)
        kospi_list.loc[idx, '감사의견_강조사항'] = ' '.join(emphs_matter_list)
        kospi_list.loc[idx, '감사의견_핵심감사사항'] = ' '.join(core_adt_matter_list)


In [10]:
kospi_list

,종목코드,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,사업자등록번호,주소,업종코드,설립일,감사의견,감사의견_강조사항,감사의견_핵심감사사항
고유번호,,,,,,,,,,,,,,,,
00155212,22100,포스코DX,"건축기술, 엔지니어링 및 관련 기술 서비스업",EIC 자동화 엔지니어링 및 IT서비스,45293,12월,정덕균,http://www.poscodx.com/kor/,경상북도,2198100428,경상북도 포항시 남구 호동로 68,72129,19891118,적정 적정 적정,해당사항 없음 해당사항 없음 해당사항 없음,- 미청구공사금액 회수가능성\n- 공사손실충당금 설정 검토\n- 법인세 및 이연법인...
01311408,450080,에코프로머티,일차전지 및 축전지 제조업,배터리 양극재용 핵심소재 하이니켈 전구체,45247,12월,김병훈,http://ecopromaterials.com/,경상북도,8108800647,경상북도 포항시 북구 흥해읍 영일만산단남로75번길 15,28202,20170426,None,None,None
01515323,373220,LG에너지솔루션,일차전지 및 축전지 제조업,"2차전지 (소형,ESS,자동차전지)",44588,12월,김동명,NaN,서울특별시,8518102050,"서울특별시 영등포구 여의대로 108, 타워 1",28202,20201201,적정 적정,SK온(주)의 물적분할 전 회사인 SK이노베이션(주)과 체결한 합의 계약에 따라 S...,해당사항 없음 해당사항 없음
01596425,402340,SK스퀘어,기타 금융업,지주회사,44529,12월,박성하,http://www.sksquare.com,서울특별시,5408102301,서울특별시 중구 을지로 65 SK T타워,64992,20211102,적정,,(연결재무제표) 보안서비스 현금창출단위영업권 손상평가(별도재무제표) 종속기업 주식에...
01244601,377300,카카오페이,금융 지원 서비스업,"간편결제, 간편송금 및 기타 금융서비스",44503,12월,신원근,http://www.kakaopay.com,경기도,5278800686,경기도 성남시 분당구 판교역로 166 카카오페이판교아지트 B동 15층,66199,20170403,적정의견 적정의견 적정의견,COVID-19 영향으로 인한 불확실성 COVID-19 영향으로 인한 불확실성,온라인 결제수익의 인식 해당사항 없음 해당사항 없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00113207,1440,대한전선,절연선 및 케이블 제조업,"전력선,통신케이블,적산계기,스텐레스압연제품,광케이블,초고압선,알루미늄 제조,도매/전기공사",25199,12월,송종민,http://www.taihan.com,경기도,1198110062,경기도 안양시 동안구 관양동 1746-2 대한스마트타워,28302,19550221,적정 적정 적정,,이연법인세자산의 실현가능성 평가\n- 이사회에 보고된 최근 사업계획 및 과거 실적을...
00160047,240,한국앤컴퍼니,기타 금융업,비금융지주회사,25199,12월,"조현범, 안종선(각자 대표이사)",http://www.hankook-technologygroup.com,경기도,2208102784,경기도 성남시 분당구 판교로 286 -,64992,19410510,적정 적정 적정,해당사항 없음 COVID-19 관련\n경제환경의 불확실성 해당사항 없음,관계기업투자 손상평가 관계기업투자 손상평가 관계기업투자 손상평가
00113526,3490,대한항공,항공 여객 운송업,"여객운송,화물운송,항공기 제조,판매,정비수리/기내식 제조판매,면세품 판매",24184,12월,"조원태, 우기홍",http://www.koreanair.com,서울특별시,1108114794,서울특별시 강서구 하늘길 260,511,19620619,적정 적정 적정,COVID-19로 인한\n영업환경의 불확실성 계속기업의 중요한\n불확실성 - 전기오...,이연수익의\n인식 항공운송자산집단 손상 유형자산손상


### Excel로 추출

In [ ]:
kospi_list.to_excel('kospi_report.xlsx')
kospi_list.info()

### URL 1,2,3 합치기

In [9]:
import json
import requests
import pandas as pd

key = "022d222dfd753fee87cfafad7fe54169c4d72eb9"
year = 2023

# DataFrame 초기화
kospi_list = pd.DataFrame(index=kospi_list.index)

# 최대주주 정보
kospi_list['최대주주_성명'], kospi_list['최대주주_관계'], kospi_list['최대주주_주식종류'] = None, None, None
kospi_list['최대주주_기초주식수'], kospi_list['최대주주_기초주식지분율'] = None, None
kospi_list['최대주주_기말주식수'], kospi_list['최대주주_기말주식지분율'] = None, None

# 소액주주 정보
kospi_list['소액주주수'], kospi_list['주주수'], kospi_list['소액주주비율'] = None, None, None
kospi_list['소액주주_주식수'], kospi_list['총주식수'], kospi_list['소액주주_주식지분율'] = None, None, None

# 감사 정보
kospi_list['감사의견'], kospi_list['감사의견_강조사항'], kospi_list['감사의견_핵심감사사항'] = None, None, None

for idx in kospi_list.index:
    # 최대주주 정보 가져오기
    res_url_1 = requests.get(f'https://opendart.fss.or.kr/api/hyslrSttus.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011',  
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'})    
    json_reports = json.loads(res_url_1.text)
    if json_reports.get('list'):
        for company_report in json_reports['list']:
            if company_report.get('relate') == "최대주주":
                # 최대주주 정보만 가져오기
                kospi_list.loc[idx, '최대주주_성명'] = company_report.get('nm', None)
                kospi_list.loc[idx, '최대주주_관계'] = company_report.get('relate', None)
                kospi_list.loc[idx, '최대주주_주식종류'] = company_report.get('stock_knd', None)
                kospi_list.loc[idx, '최대주주_기초주식수'] = company_report.get('bsis_posesn_stock_co', None)
                kospi_list.loc[idx, '최대주주_기초주식지분율'] = company_report.get('bsis_posesn_stock_qota_rt', None)
                kospi_list.loc[idx, '최대주주_기말주식수'] = company_report.get('trmend_posesn_stock_co', None)
                kospi_list.loc[idx, '최대주주_기말주식지분율'] = company_report.get('trmend_posesn_stock_qota_rt', None)

    # 소액주주 정보 가져오기
    res_url_2 = requests.get(f'https://opendart.fss.or.kr/api/mrhlSttus.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011',  
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'})    
    json_reports = json.loads(res_url_2.text)
    if json_reports.get('list'):
        for company_report in json_reports['list']:
            kospi_list.loc[idx, '소액주주수'] = company_report.get('shrholdr_co', None)
            kospi_list.loc[idx, '주주수'] = company_report.get('shrholdr_tot_co', None)
            kospi_list.loc[idx, '소액주주비율'] = company_report.get('shrholdr_rate', None)
            kospi_list.loc[idx, '소액주주_주식수'] = company_report.get('hold_stock_co', None)
            kospi_list.loc[idx, '총주식수'] = company_report.get('stock_tot_co', None)
            kospi_list.loc[idx, '소액주주_주식지분율'] = company_report.get('hold_stock_rate', None)

    # 감사 정보 가져오기
    res_url_3 = requests.get(f'https://opendart.fss.or.kr/api/accnutAdtorNmNdAdtOpinion.json?crtfc_key={key}&corp_code={idx}&bsns_year={year}&reprt_code=11011', headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    })    
    json_reports = json.loads(res_url_3.text)
    if json_reports.get('list'):
        adt_opinion_list, emphs_matter_list, core_adt_matter_list = [], [], []
        for company_report in json_reports['list']:
            adt_opinion_list.append(company_report.get('adt_opinion', ''))
            emphs_matter_list.append(company_report.get('emphs_matter', ''))
            core_adt_matter_list.append(company_report.get('core_adt_matter', ''))

        # 리스트에 있는 값을 하나로 합쳐서 DataFrame에 할당
        kospi_list.loc[idx, '감사의견'] = ' '.join(adt_opinion_list)
        kospi_list.loc[idx, '감사의견_강조사항'] = ' '.join(emphs_matter_list)
        kospi_list.loc[idx, '감사의견_핵심감사사항'] = ' '.join(core_adt_matter_list)

print(kospi_list)


            최대주주_성명 최대주주_관계 최대주주_주식종류   최대주주_기초주식수 최대주주_기초주식지분율  최대주주_기말주식수  \
고유번호                                                                          
00155212     포스코홀딩스    최대주주       보통주   99,403,282        65.38  99,403,282   
01311408  주식회사 에코프로    최대주주       보통주   30,559,067        52.78  30,559,067   
01515323       None    None      None         None         None        None   
01596425        SK㈜    최대주주  의결권있는 주식   42,452,202        30.01  42,452,202   
01244601       ㈜카카오    최대주주       보통주   62,351,920        46.88  62,351,920   
...             ...     ...       ...          ...          ...         ...   
00113207  주식회사 호반산업    최대주주       보통주  499,044,607        40.10  49,904,460   
00160047       None    None      None         None         None        None   
00113526       None    None      None         None         None        None   
00145109       None    None      None         None         None        None   
00113410       None    None      None         None  

In [10]:
kospi_list

,최대주주_성명,최대주주_관계,최대주주_주식종류,최대주주_기초주식수,최대주주_기초주식지분율,최대주주_기말주식수,최대주주_기말주식지분율,소액주주수,주주수,소액주주비율,소액주주_주식수,총주식수,소액주주_주식지분율,감사의견,감사의견_강조사항,감사의견_핵심감사사항
고유번호,,,,,,,,,,,,,,,,
00155212,포스코홀딩스,최대주주,보통주,"99,403,282",65.38,"99,403,282",65.38,"235,155","235,161",99.99%,"49,815,255","151,822,519",32.81%,적정 적정 적정,해당사항 없음 해당사항 없음 해당사항 없음,- 프로젝트 자료 및 데이터 간 정합성 검토\n- 매출채권 회수가능성 검토 - 총계...
01311408,주식회사 에코프로,최대주주,보통주,"30,559,067",52.78,"30,559,067",44.30,"138,512","138,521",99.99%,"13,085,653","68,985,218",18.97%,적정 적정 적정,경영진이 코로나바이러스감염증-19의 확산이 기업의 생산 능력과 고객에 대한 주문...,"당기말 현재 회사의 유형자산은\n 430,746백만원으로 재무제표에 유의적인 금액입..."
01515323,None,None,None,None,None,None,None,"868,305","868,365",99.99%,"29,538,819","234,000,000",12.62%,적정 적정 적정,해당사항 없음 해당사항 없음 SK온(주)의 물적분할 전 회사인 SK이노베이션(주)과...,GM 리콜 충당부채 (1) GM 리콜 충당부채\n(2) 종속기업 투자주식 손상검토 ...
01596425,SK㈜,최대주주,의결권있는 주식,"42,452,202",30.01,"42,452,202",30.01,"134,173","134,218",99.97%,"49,292,177","138,981,036",35.47%,,,
01244601,㈜카카오,최대주주,보통주,"62,351,920",46.88,"62,351,920",46.44,"275,859","275,862",99.99%,"21,461,216","134,265,066",15.98%,적정의견 적정의견 적정의견,해당사항 없음 해당사항 없음 COVID-19 영향으로 인한 불확실성,(별도재무제표)\n1. 결제서비스 수익인식\n2. 카카오페이증권 종속기업투자주식의 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
00113207,주식회사 호반산업,최대주주,보통주,"499,044,607",40.10,"49,904,460",40.10,"236,776","236,778",99.99%,"72,492,641","123,675,213",58.62%,적정 적정 적정,,이연법인세자산의 실현가능성 평가\n- 이사회에 보고된 최근 사업계획 및 과거 실적을...
00160047,None,None,None,None,None,None,None,"22,372","22,394",99.90%,"17,096,933","94,935,240",18.01%,적정 적정 적정,해당사항 없음 해당사항 없음 해당사항 없음,관계기업투자 손상평가 관계기업투자 손상평가 관계기업투자 손상평가
00113526,None,None,None,None,None,None,None,"718,523","726,695",98.88%,"222,111,851","369,331,455",60.14%,적정\n 예외사항 없음\n 예외사항 없음\n 예외사항 없음 적정\n 예외사항 없음\...,해당사항 없음 해당사항 없음 COVID-19로 인한\n영업환경의 불확실성,이연수익의 인식 이연수익의 인식 이연수익의 인식


In [11]:
kospi_list.to_excel('kospi_info.xlsx')
kospi_list.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 00155212 to 00113410
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   최대주주_성명       77 non-null     object
 1   최대주주_관계       77 non-null     object
 2   최대주주_주식종류     77 non-null     object
 3   최대주주_기초주식수    77 non-null     object
 4   최대주주_기초주식지분율  77 non-null     object
 5   최대주주_기말주식수    77 non-null     object
 6   최대주주_기말주식지분율  77 non-null     object
 7   소액주주수         200 non-null    object
 8   주주수           200 non-null    object
 9   소액주주비율        200 non-null    object
 10  소액주주_주식수      200 non-null    object
 11  총주식수          200 non-null    object
 12  소액주주_주식지분율    200 non-null    object
 13  감사의견          200 non-null    object
 14  감사의견_강조사항     200 non-null    object
 15  감사의견_핵심감사사항   200 non-null    object
dtypes: object(16)
memory usage: 34.7+ KB
